In [58]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [59]:
dataset = 'RNJ2017'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/Table 1.csv', header=5)
print('Raw data loaded.')
data

Loading raw data for RNJ2017 ...
Raw data loaded.


,Protein_Peptide_phosphosite,Protein_localized phosphosite,Gene,Protein Accession No.,Log2 ratio\nTsup:Tstim_D1,Log2 ratio \nTsup:Tstim_D2,Log2 ratio\nTsup:Tstim_D3,Mean Log2 ratio\nTsup:Tstim,P Tsup:Tstim,Log2 ratio \nTstim:Trest_D1,Log2 ratio\nTstim:Trest_D2,Log2 ratio\nTstim:Trest_D3,Mean Log2 ratio\nTstim:Trest,P Tstim:Trest,Log2 ratio\nTsup:Trest_D1,Log2 ratio\nTsup:Trest_D2,Log2 ratio\nTsup:Trest_D3,Mean Log2 ratio\nTsup:Trest,P Tsup:Trest
0,Q2M2I8_ILSDVTHSAVFGVPASK_S(3),Q2M2I8_S637,AAK1,Q2M2I8,0.103185,-0.295044,-0.203012,-0.132,0.59378,-0.300915,0.454669,-0.594960,-0.147,0.55229,-0.197731,0.159625,-0.797972,-0.279,0.27605
1,Q2M2I8_VGSLTPPSSPK_T(5),Q2M2I8_T620,AAK1,Q2M2I8,0.022814,-0.623506,-0.301903,-0.301,0.16432,0.024361,0.756067,0.101752,0.294,0.17275,0.047176,0.132561,-0.200151,-0.007,0.97287
2,Q2M2I8_VGSLTPPSSPK_T(5)_S(9),Q2M2I8_T620_S624,AAK1,Q2M2I8,0.043631,-0.219345,-0.297923,-0.158,0.32497,0.082995,0.143815,0.061059,0.096,0.54013,0.126626,-0.075531,-0.236863,-0.062,0.69009
3,Q2M2I8_VQTTPPPAVQGQK_T(4),Q2M2I8_T606,AAK1,Q2M2I8,-0.011927,NaN,-0.220072,-0.127,0.59706,-0.243325,NaN,-0.226497,0.061,0.79881,-0.255251,NaN,-0.446569,-0.066,0.78093
4,Q2M2I8_VGSLTPPSSPK_S(8)_S(9),Q2M2I8_S623_S624,AAK1,Q2M2I8,NaN,-0.328536,-0.225803,-0.253,0.11552,NaN,0.269203,0.145844,0.138,0.35864,NaN,-0.059332,-0.079959,-0.115,0.43982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,Q96NB3_KEEENADSDDEGELQDLLSQDWR_S(8),Q96NB3_S351,ZNF830,Q96NB3,-0.154632,-0.057478,-0.346595,-0.186,0.31659,-0.115795,0.303887,0.456850,0.215,0.25343,-0.270427,0.246408,0.110255,0.029,0.87231
1013,O95218_YNLDASEEEDSNK_S(6),O95218_S188,ZRANB2,O95218,0.240996,-0.374288,0.023289,-0.037,0.84265,-0.026681,0.514704,-0.038182,0.150,0.42780,0.214316,0.140416,-0.014893,0.113,0.54495
1014,O95218_ENVEYIEREESDGEYDEFGRK_S(11),O95218_S120,ZRANB2,O95218,0.566342,NaN,-0.341385,0.024,0.92505,-0.626462,NaN,0.429223,0.089,0.72886,-0.060120,NaN,0.087838,0.113,0.66062
1015,O95218_YNLDASEEEDSNKK_S(6),O95218_S188,ZRANB2,O95218,-0.337377,NaN,-0.591503,-0.346,0.06495,0.427329,NaN,0.413864,0.321,0.08220,0.089953,NaN,-0.177638,-0.025,0.87609


In [60]:
print(data.columns)

Index(['Protein_Peptide_phosphosite', 'Protein_localized phosphosite', 'Gene',
       'Protein Accession No.', 'Log2 ratio\nTsup:Tstim_D1',
       'Log2 ratio \nTsup:Tstim_D2', 'Log2 ratio\nTsup:Tstim_D3',
       'Mean Log2 ratio\nTsup:Tstim', 'P Tsup:Tstim',
       'Log2 ratio \nTstim:Trest_D1', 'Log2 ratio\nTstim:Trest_D2',
       'Log2 ratio\nTstim:Trest_D3', 'Mean Log2 ratio\nTstim:Trest',
       'P Tstim:Trest', 'Log2 ratio\nTsup:Trest_D1',
       'Log2 ratio\nTsup:Trest_D2', 'Log2 ratio\nTsup:Trest_D3',
       'Mean Log2 ratio\nTsup:Trest', 'P Tsup:Trest'],
      dtype='object')


In [61]:
data.columns = data.columns.str.strip()

In [62]:
data['Sequence window'] = data['Protein_Peptide_phosphosite'].str.extract(r'^[^_]+_([^_]+)')

# Extract the amino acid and position from Protein_localized phosphosite
data['Amino acid'] = data['Protein_localized phosphosite'].str.extract(r'_([A-Z])\d+')
data['Position'] = data['Protein_localized phosphosite'].str.extract(r'_[A-Z](\d+)')
data = data[data['Protein_localized phosphosite'].str.count('_') == 1]

In [63]:
print(data[['Protein_localized phosphosite', 'Amino acid', 'Position']].head())


  Protein_localized phosphosite Amino acid Position
0                   Q2M2I8_S637          S      637
1                   Q2M2I8_T620          T      620
3                   Q2M2I8_T606          T      606
7                   Q8NE71_S140          S      140
8                   Q8NE71_S109          S      109


In [64]:
# filter data
data['Sequence window'] = data['Sequence window'].str.replace('_', '')
data

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_998/1172424322.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sequence window'] = data['Sequence window'].str.replace('_', '')


,Protein_Peptide_phosphosite,Protein_localized phosphosite,Gene,Protein Accession No.,Log2 ratio\nTsup:Tstim_D1,Log2 ratio \nTsup:Tstim_D2,Log2 ratio\nTsup:Tstim_D3,Mean Log2 ratio\nTsup:Tstim,P Tsup:Tstim,Log2 ratio \nTstim:Trest_D1,...,Mean Log2 ratio\nTstim:Trest,P Tstim:Trest,Log2 ratio\nTsup:Trest_D1,Log2 ratio\nTsup:Trest_D2,Log2 ratio\nTsup:Trest_D3,Mean Log2 ratio\nTsup:Trest,P Tsup:Trest,Sequence window,Amino acid,Position
0,Q2M2I8_ILSDVTHSAVFGVPASK_S(3),Q2M2I8_S637,AAK1,Q2M2I8,0.103185,-0.295044,-0.203012,-0.132,0.59378,-0.300915,...,-0.147,0.55229,-0.197731,0.159625,-0.797972,-0.279,0.27605,ILSDVTHSAVFGVPASK,S,637
1,Q2M2I8_VGSLTPPSSPK_T(5),Q2M2I8_T620,AAK1,Q2M2I8,0.022814,-0.623506,-0.301903,-0.301,0.16432,0.024361,...,0.294,0.17275,0.047176,0.132561,-0.200151,-0.007,0.97287,VGSLTPPSSPK,T,620
3,Q2M2I8_VQTTPPPAVQGQK_T(4),Q2M2I8_T606,AAK1,Q2M2I8,-0.011927,NaN,-0.220072,-0.127,0.59706,-0.243325,...,0.061,0.79881,-0.255251,NaN,-0.446569,-0.066,0.78093,VQTTPPPAVQGQK,T,606
7,Q8NE71_GGNVFAALIQDQSEEEEEEEK_S(13),Q8NE71_S140,ABCF1,Q8NE71,0.477370,0.079655,-1.440280,-0.294,0.50522,0.276106,...,0.632,0.17600,0.753477,-0.142205,0.402837,0.338,0.44671,GGNVFAALIQDQSEEEEEEEK,S,140
8,Q8NE71_LSVPTSDEEDEVPAPKPR_S(6),Q8NE71_S109,ABCF1,Q8NE71,0.195185,0.214832,-1.730006,-0.440,0.29918,0.274203,...,0.535,0.21552,0.469388,0.095287,-0.280488,0.095,0.81588,LSVPTSDEEDEVPAPKPR,S,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,Q96NB3_KEEENADSDDEGELQDLLSQDWR_S(8),Q96NB3_S351,ZNF830,Q96NB3,-0.154632,-0.057478,-0.346595,-0.186,0.31659,-0.115795,...,0.215,0.25343,-0.270427,0.246408,0.110255,0.029,0.87231,KEEENADSDDEGELQDLLSQDWR,S,351
1013,O95218_YNLDASEEEDSNK_S(6),O95218_S188,ZRANB2,O95218,0.240996,-0.374288,0.023289,-0.037,0.84265,-0.026681,...,0.150,0.42780,0.214316,0.140416,-0.014893,0.113,0.54495,YNLDASEEEDSNK,S,188
1014,O95218_ENVEYIEREESDGEYDEFGRK_S(11),O95218_S120,ZRANB2,O95218,0.566342,NaN,-0.341385,0.024,0.92505,-0.626462,...,0.089,0.72886,-0.060120,NaN,0.087838,0.113,0.66062,ENVEYIEREESDGEYDEFGRK,S,120
1015,O95218_YNLDASEEEDSNKK_S(6),O95218_S188,ZRANB2,O95218,-0.337377,NaN,-0.591503,-0.346,0.06495,0.427329,...,0.321,0.08220,0.089953,NaN,-0.177638,-0.025,0.87609,YNLDASEEEDSNKK,S,188


In [65]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 

In [66]:
data = match_seq_to_genename(data, 'Sequence window')

ILSDVTHSAVFGVPASK
Match found for sequence: ID: sp|Q2M2I8|AAK1_HUMAN
Name: sp|Q2M2I8|AAK1_HUMAN
Description: sp|Q2M2I8|AAK1_HUMAN AP2-associated protein kinase 1 OS=Homo sapiens OX=9606 GN=AAK1 PE=1 SV=3
Number of features: 0
Seq('MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAE...IDL')
Gene name match: <re.Match object; span=(77, 84), match='GN=AAK1'>
Match found: ILSDVTHSAVFGVPASK -> AAK1
VGSLTPPSSPK
Match found for sequence: ID: sp|Q2M2I8|AAK1_HUMAN
Name: sp|Q2M2I8|AAK1_HUMAN
Description: sp|Q2M2I8|AAK1_HUMAN AP2-associated protein kinase 1 OS=Homo sapiens OX=9606 GN=AAK1 PE=1 SV=3
Number of features: 0
Seq('MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAE...IDL')
Gene name match: <re.Match object; span=(77, 84), match='GN=AAK1'>
Match found: VGSLTPPSSPK -> AAK1
VQTTPPPAVQGQK
Match found for sequence: ID: sp|Q2M2I8|AAK1_HUMAN
Name: sp|Q2M2I8|AAK1_HUMAN
Description: sp|Q2M2I8|AAK1_HUMAN AP2-associated protein kinase 1 OS=Homo sapiens OX=9606 GN=AAK1 PE=1 SV=3
Number of fe

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_998/1670126104.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['GeneName'] = dataset[seq_column].map(gene_dict)


In [67]:
data['Phosphosite'] = data['Amino acid'].astype(str) + '(' + data['Position'].astype(str) + ')'
data

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_998/2381489477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Phosphosite'] = data['Amino acid'].astype(str) + '(' + data['Position'].astype(str) + ')'


,Protein_Peptide_phosphosite,Protein_localized phosphosite,Gene,Protein Accession No.,Log2 ratio\nTsup:Tstim_D1,Log2 ratio \nTsup:Tstim_D2,Log2 ratio\nTsup:Tstim_D3,Mean Log2 ratio\nTsup:Tstim,P Tsup:Tstim,Log2 ratio \nTstim:Trest_D1,...,Log2 ratio\nTsup:Trest_D1,Log2 ratio\nTsup:Trest_D2,Log2 ratio\nTsup:Trest_D3,Mean Log2 ratio\nTsup:Trest,P Tsup:Trest,Sequence window,Amino acid,Position,GeneName,Phosphosite
0,Q2M2I8_ILSDVTHSAVFGVPASK_S(3),Q2M2I8_S637,AAK1,Q2M2I8,0.103185,-0.295044,-0.203012,-0.132,0.59378,-0.300915,...,-0.197731,0.159625,-0.797972,-0.279,0.27605,ILSDVTHSAVFGVPASK,S,637,AAK1,S(637)
1,Q2M2I8_VGSLTPPSSPK_T(5),Q2M2I8_T620,AAK1,Q2M2I8,0.022814,-0.623506,-0.301903,-0.301,0.16432,0.024361,...,0.047176,0.132561,-0.200151,-0.007,0.97287,VGSLTPPSSPK,T,620,AAK1,T(620)
3,Q2M2I8_VQTTPPPAVQGQK_T(4),Q2M2I8_T606,AAK1,Q2M2I8,-0.011927,NaN,-0.220072,-0.127,0.59706,-0.243325,...,-0.255251,NaN,-0.446569,-0.066,0.78093,VQTTPPPAVQGQK,T,606,AAK1,T(606)
7,Q8NE71_GGNVFAALIQDQSEEEEEEEK_S(13),Q8NE71_S140,ABCF1,Q8NE71,0.477370,0.079655,-1.440280,-0.294,0.50522,0.276106,...,0.753477,-0.142205,0.402837,0.338,0.44671,GGNVFAALIQDQSEEEEEEEK,S,140,ABCF1,S(140)
8,Q8NE71_LSVPTSDEEDEVPAPKPR_S(6),Q8NE71_S109,ABCF1,Q8NE71,0.195185,0.214832,-1.730006,-0.440,0.29918,0.274203,...,0.469388,0.095287,-0.280488,0.095,0.81588,LSVPTSDEEDEVPAPKPR,S,109,ABCF1,S(109)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,Q96NB3_KEEENADSDDEGELQDLLSQDWR_S(8),Q96NB3_S351,ZNF830,Q96NB3,-0.154632,-0.057478,-0.346595,-0.186,0.31659,-0.115795,...,-0.270427,0.246408,0.110255,0.029,0.87231,KEEENADSDDEGELQDLLSQDWR,S,351,ZNF830,S(351)
1013,O95218_YNLDASEEEDSNK_S(6),O95218_S188,ZRANB2,O95218,0.240996,-0.374288,0.023289,-0.037,0.84265,-0.026681,...,0.214316,0.140416,-0.014893,0.113,0.54495,YNLDASEEEDSNK,S,188,ZRANB2,S(188)
1014,O95218_ENVEYIEREESDGEYDEFGRK_S(11),O95218_S120,ZRANB2,O95218,0.566342,NaN,-0.341385,0.024,0.92505,-0.626462,...,-0.060120,NaN,0.087838,0.113,0.66062,ENVEYIEREESDGEYDEFGRK,S,120,ZRANB2,S(120)
1015,O95218_YNLDASEEEDSNKK_S(6),O95218_S188,ZRANB2,O95218,-0.337377,NaN,-0.591503,-0.346,0.06495,0.427329,...,0.089953,NaN,-0.177638,-0.025,0.87609,YNLDASEEEDSNKK,S,188,ZRANB2,S(188)


In [68]:
data.columns = data.columns.str.replace('\n', ' ', regex=True).str.strip()


In [69]:
keepcols = ['Phosphosite', 'GeneName'] + [col for col in data.columns if col.startswith(('Log2 ratio Tsup:Tstim', 'Log2 ratio Tstim:Trest', 'Log2 ratio Tsup:Trest'))]
data = data[keepcols]
data

,Phosphosite,GeneName,Log2 ratio Tsup:Tstim_D1,Log2 ratio Tsup:Tstim_D3,Log2 ratio Tstim:Trest_D2,Log2 ratio Tstim:Trest_D3,Log2 ratio Tsup:Trest_D1,Log2 ratio Tsup:Trest_D2,Log2 ratio Tsup:Trest_D3
0,S(637),AAK1,0.103185,-0.203012,0.454669,-0.594960,-0.197731,0.159625,-0.797972
1,T(620),AAK1,0.022814,-0.301903,0.756067,0.101752,0.047176,0.132561,-0.200151
3,T(606),AAK1,-0.011927,-0.220072,NaN,-0.226497,-0.255251,NaN,-0.446569
7,S(140),ABCF1,0.477370,-1.440280,-0.221860,1.843117,0.753477,-0.142205,0.402837
8,S(109),ABCF1,0.195185,-1.730006,-0.119545,1.449518,0.469388,0.095287,-0.280488
...,...,...,...,...,...,...,...,...,...
1012,S(351),ZNF830,-0.154632,-0.346595,0.303887,0.456850,-0.270427,0.246408,0.110255
1013,S(188),ZRANB2,0.240996,0.023289,0.514704,-0.038182,0.214316,0.140416,-0.014893
1014,S(120),ZRANB2,0.566342,-0.341385,NaN,0.429223,-0.060120,NaN,0.087838
1015,S(188),ZRANB2,-0.337377,-0.591503,NaN,0.413864,0.089953,NaN,-0.177638


In [70]:
ratio_columns = [col for col in data.columns if col.startswith(('Log2 ratio Tsup:Tstim', 'Log2 ratio Tstim:Trest', 'Log2 ratio Tsup:Trest'))]
data[ratio_columns] = data[ratio_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_998/2200052052.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[ratio_columns] = data[ratio_columns].apply(pd.to_numeric, errors='coerce')


In [71]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

Phosphosite IDs created.
Phosphosite IDs created.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_998/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


In [72]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case = False)]
    data = data[~data.phosphosite_ID.str.contains(';', case = False)] # remove rows containing ';' in phosphosite_ID column
    data = data[~data.phosphosite_ID.str.contains('-', case = False)] # remove rows containing '-' in phosphosite_ID column
    
    # check whether there are any phosphosites with multiple measurements
    data_grouped = data.groupby(by = 'phosphosite_ID')
    if len(data) != len(data_grouped):
        data = data_grouped.mean()
        data.reset_index(inplace=True) # reset index
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')
        
    print(data)
        
    data = data.replace([np.inf, -np.inf], np.nan)
        
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)
    return data


In [73]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

Phosphosites with multiple measurements have been averaged
    phosphosite_ID  Log2 ratio Tsup:Tstim_D1  Log2 ratio Tsup:Tstim_D3  \
0      AAK1_S(637)                  0.103185                 -0.203012   
1      AAK1_T(606)                 -0.011927                 -0.220072   
2      AAK1_T(620)                  0.022814                 -0.301903   
3     ABCF1_S(109)                  0.347084                 -1.717925   
4     ABCF1_S(140)                  0.477370                 -1.440280   
..             ...                       ...                       ...   
606  ZMYND8_S(756)                  0.129402                 -1.109469   
607  ZNF830_S(351)                 -0.154632                 -0.346595   
608  ZRANB2_S(120)                  0.566342                 -0.341385   
609  ZRANB2_S(188)                 -0.048190                 -0.284107   
610     ZYX_S(259)                 -0.431662                 -1.168454   

     Log2 ratio Tstim:Trest_D2  Log2 ratio Tstim:Tre

,phosphosite_ID,Log2 ratio Tsup:Tstim_D1,Log2 ratio Tsup:Tstim_D3,Log2 ratio Tstim:Trest_D2,Log2 ratio Tstim:Trest_D3,Log2 ratio Tsup:Trest_D1,Log2 ratio Tsup:Trest_D2,Log2 ratio Tsup:Trest_D3
0,AAK1_S(637),0.103185,-0.203012,0.454669,-0.594960,-0.197731,0.159625,-0.797972
1,AAK1_T(606),-0.011927,-0.220072,NaN,-0.226497,-0.255251,NaN,-0.446569
2,AAK1_T(620),0.022814,-0.301903,0.756067,0.101752,0.047176,0.132561,-0.200151
3,ABCF1_S(109),0.347084,-1.717925,-0.119545,1.423855,0.537594,0.095287,-0.294070
4,ABCF1_S(140),0.477370,-1.440280,-0.221860,1.843117,0.753477,-0.142205,0.402837
...,...,...,...,...,...,...,...,...
606,ZMYND8_S(756),0.129402,-1.109469,0.219896,-0.038464,0.141598,0.251296,-1.147934
607,ZNF830_S(351),-0.154632,-0.346595,0.303887,0.456850,-0.270427,0.246408,0.110255
608,ZRANB2_S(120),0.566342,-0.341385,NaN,0.429223,-0.060120,NaN,0.087838
609,ZRANB2_S(188),-0.048190,-0.284107,0.514704,0.187841,0.152134,0.140416,-0.096266


In [74]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/RNJ2017.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data) 

RNJ2017 has been saved to CSV successfully!     phosphosite_ID  Log2 ratio Tsup:Tstim_D1  Log2 ratio Tsup:Tstim_D3  \
0      AAK1_S(637)                  0.103185                 -0.203012   
1      AAK1_T(606)                 -0.011927                 -0.220072   
2      AAK1_T(620)                  0.022814                 -0.301903   
3     ABCF1_S(109)                  0.347084                 -1.717925   
4     ABCF1_S(140)                  0.477370                 -1.440280   
..             ...                       ...                       ...   
606  ZMYND8_S(756)                  0.129402                 -1.109469   
607  ZNF830_S(351)                 -0.154632                 -0.346595   
608  ZRANB2_S(120)                  0.566342                 -0.341385   
609  ZRANB2_S(188)                 -0.048190                 -0.284107   
610     ZYX_S(259)                 -0.431662                 -1.168454   

     Log2 ratio Tstim:Trest_D2  Log2 ratio Tstim:Trest_D3  \
0     